<p><font size=5 color="#210F6E"><em><u><center>Fast Data Analysis for English tweets</center></u></em></font><p>
<br>
<font face="Gill Sans" color="#261D7A" size="3.2">As we did before with #PrimaireLeDébat tweets we recovered tweets having the hashtag #Lalaland in the TwitteR_Recover repository</font><br><br>
<font color="#206B50" size = 3><center>**DATA CLEANING**</center></font>

In [ ]:
yourPath = paste0(getwd(),"/data/Lalaland.csv")

tweets = read.csv(yourPath, encoding="UTF-8")
print(dim(tweets))

tweets <- subset(tweets, select=-c(replyToSN,replyToUID, replyToSID, latitude, longitude, favorited))
#head(tweets, n=5)

In [ ]:
clean_text = function(x)
{
    #To convert the text in lowercase
    try.error = function(z)
    {
        y = NA
        try_error = tryCatch(tolower(z), error=function(e) e)
            if (!inherits(try_error, "error"))
                y = tolower(z)
                return(y)
    }
            
    x = sapply(x, try.error)
    
    #Delete #equivalent
    x = gsub("#primaire\\w+ *", "", x)
            
     #remove all links starting by http
    x = gsub('http\\S+\\s*', '', x)
            
    # replace apostrophes
    x = gsub("'", " ", x)

    # remove punctuation except @, #, _, -
    x = gsub("@", "AAAAAAAAAAA", x)
    x = gsub("#", "BBBBBBBBBBB", x)
    x = gsub("_", "CCCCCCCCCCC", x)
    x = gsub("-", "DDDDDDDDDDD", x)
    x = gsub("[[:punct:]]", " ", x)
    x = gsub("AAAAAAAAAAA", "@", x)
    x = gsub("BBBBBBBBBBB", "#", x)
    x = gsub("CCCCCCCCCCC", "_", x)
    x = gsub("DDDDDDDDDDD", "-", x)
            
    # correcting the spaces after the conserved @
    x = gsub("@ ", "@", x)
            
    # correcting the spaces after the conserved _
    x = gsub("_ ", "_", x)
            
    # correcting the spaces after the conserved -
    x = gsub("- ", "-", x)
    
    # remove numbers/Digits
    x = gsub("[[:digit:]]", "", x)
    
    # remove tabs
    x = gsub("[ |\t]{2,}", " ", x)
            
    # remove blank spaces at the beginning/end
    x = gsub("^ ", "", x)  
    x = gsub(" $", "", x)
    x = gsub("'", "", x)    
    
    # As we have already a column indicating if the tweet is a retweet or not 
    # we can remove "RT @xxx" in the tweet header
    x = gsub("rt @\\w+ *", "", x)
    x = gsub('\\b\\w{1,3}\\s','', x)
            
    # remove double spaces
    x = gsub("  ", " ", x)
    x = gsub("  ", " ", x)
    return(x)
}
                             
tweets$text <- clean_text(tweets$text)
head(tweets)

In [ ]:
sum(is.na(tweets$text))

In [ ]:
tweets = na.omit(tweets)
tweets$X <- NULL
nrow(tweets)

In [ ]:
col = tweets$text
at.pattern = "@\\w+ *"
have.at = grep(x = col, pattern = at.pattern)
at.matches = gregexpr(pattern = at.pattern,
                        text = col[have.at])
extracted.at = regmatches(x = col[have.at], m = at.matches)

# most frequent words
mfw = sort(unlist(extracted.at), decreasing=TRUE)
mfw = gsub(" ", "", mfw)
top40_user_called = sort(table(unlist(mfw)), decreasing=TRUE)
head(top40_user_called, n = 40)

In [ ]:
top5 = head(top40_user_called, n = 5)
barplot(sort(top5), border=NA, las=2, main="Top 5 most frequent word", cex.main=1, horiz=TRUE, col= "darkblue", cex.names=0.5)

In [ ]:
at.pattern = "[a-zA-Z]\\w+ *"
have.at = grep(x = col, pattern = at.pattern)
at.matches = gregexpr(pattern = at.pattern,
                        text = col[have.at])
extracted.at = regmatches(x = col[have.at], m = at.matches)

# most frequent words
mfw = sort(unlist(extracted.at), decreasing=TRUE)
mfw = gsub(" ", "", mfw)
w = sort(table(unlist(mfw)), decreasing=TRUE)

number_Top = function(number){
  topWord = head(w, n = number)
  topWord = sort(topWord, decreasing=FALSE) 
  return(topWord)
}

top15 = number_Top(15)
barplot(top15, border=NA, las=2, main="Top 15 most frequent word", cex.main=1, horiz=TRUE, col= "darkblue", cex.names=0.6)

Sentiment Analysis

In [ ]:
#tweets = tweets[-(483), ]
length(tweets$text)
length(tweets$score)
tweets$score = NA

In [ ]:
#install.packages("RSentiment") #Requiered Java 
library(RSentiment)

tweets$score = 0
for (i in 1:nrow(tweets)){
 tweets$score[i] = sum(calculate_score(tweets$text[i]))   
}

head(tweets)

In [ ]:
Global_Lalaland_Sentiment = calculate_total_presence_sentiment(tweets$text)
Global_Lalaland_Sentiment

In [ ]:
#install.packages("wordcloud")
#install.packages("SnowballC")
library(SnowballC)
library(wordcloud)

top50 = number_Top(50)
wordcloud(names(top50), top50, min.freq=2)

In [ ]:
#require(devtools)
#install_github("lchiffon/wordcloud2")
library(wordcloud2)

top100 = number_Top(100)
cat("The letterCloud function does not work on Jupyter notebook, if you are using markdown, you can uncomment this line.")
#letterCloud(top100, word = "PS", wordSize = 1)